## Import libraries

In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
import time
import random
import pandas as pd
import pickle
import numpy as np
from tqdm.auto import tqdm
from datetime import datetime
from itertools import product
import torch
from torch import nn
from typing import List, Tuple, Dict, Optional
from sklearn.preprocessing import MaxAbsScaler, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.utils.losses import SmapeLoss
from darts.dataprocessing.transformers import Scaler
from darts.metrics import smape
from darts.utils.utils import SeasonalityMode, TrendMode, ModelMode
from darts.models import *

from darts.datasets import TrafficDataset, AirPassengersDataset, AustralianTourismDataset
from ucimlrepo import fetch_ucirepo 

## Import Data
- We try multiple datasets to see which provides us with the best outcome

In [ ]:
room_df

In [ ]:
room_occupancy_estimation = fetch_ucirepo(id=864) 
room_df = room_occupancy_estimation['data']['features']
room_df = pd.concat([room_df, room_occupancy_estimation['data']['targets']], axis = 1)
room_df['Capacity %'] = room_df['Room_Occupancy_Count']/max(room_df['Room_Occupancy_Count'])

airpass_raw = AirPassengersDataset().load().pd_dataframe().reset_index()
airpass_raw.columns.name = None

airpass_raw

## Airpassenger Model

### Data preprocessing
- Scale the data
    - Make sure each data entry is $0\leq x\leq 1$

In [ ]:
## Find local minimums and plot the month
airpass_data = airpass_raw.copy()
airpass_data['LMin'] = 0
airpass_data['LMax'] = 0
passengers = []

for row, index in airpass_data.iterrows():
    passengers.append(index['#Passengers'])
    
for i in range(len(passengers)):
    if i > 0 and i < len(passengers)-1:
        if passengers[i] < passengers[i-1] and passengers[i] < passengers[i+1]:
            airpass_data.loc[i, "LMin"] = 1
        if passengers[i] > passengers[i-1] and passengers[i] > passengers[i+1]:
            airpass_data.loc[i, "LMax"] = 1

## Keep out these values to prevent any errors
to_conc = airpass_data[['Month', "LMin", "LMax"]]

## Scale train and test
scaler = MaxAbsScaler()
airpass_scaled_data = pd.DataFrame(scaler.fit_transform(np.array(airpass_data.drop(columns = ['Month', "LMin", "LMax"])))).rename(columns = {0:"# Passengers"})

## Add month to both
airpass_scaled_data = pd.concat([to_conc, airpass_scaled_data], axis = 1)
airpass_scaled_data

### Plot scaled data
- Looking at the data we see some trends
    - Cyclical in nature
    - Upward trend from 1950 onwards

In [ ]:
fig, axes = plt.subplots(nrows = 1, ncols = 1, figsize = (12, 6))
lmins = airpass_scaled_data[airpass_scaled_data['LMin'] == 1]
lmaxs = airpass_scaled_data[airpass_scaled_data['LMax'] == 1]

axes.plot(airpass_scaled_data['Month'], airpass_scaled_data['# Passengers'], label = '# Passengers')
axes.set_xlabel("Month")
axes.set_ylabel("# Passengers")
axes.set_title("Scaled # Passengers over the months")
axes.scatter(lmins['Month'], lmins['# Passengers'], color='red', label = 'Local Min')
axes.scatter(lmaxs['Month'], lmaxs['# Passengers'], color='blue', label = 'Local Max')
axes.legend()

plt.show()

### Fit a model on the data
- Invert the data

In [ ]:
## Format to TimeSeries
past = 24
future = 24
airpass_ts = TimeSeries.from_dataframe(airpass_scaled_data, "Month", "# Passengers")
airpass_ts = airpass_ts.astype('float32')

model = NHiTSModel(input_chunk_length = past, output_chunk_length = future)
model.fit(airpass_ts, verbose = 0)

## Predict on self
pred = model.predict(n = len(airpass_scaled_data['Month']), verbose = 0)

## Plot predict v train
results_df = pd.DataFrame(scaler.inverse_transform(pred.values())).rename(columns = {0:"Predicted"})
results_df = pd.concat([airpass_scaled_data['Month'], results_df, \
                        pd.DataFrame(scaler.inverse_transform(airpass_ts.values()))], axis = 1)\
                        .rename(columns = {0:"Actual"})

plt.plot(results_df['Month'], results_df['Predicted'], label = 'Predicted')
plt.plot(results_df['Month'], results_df['Actual'], label = 'Actual')
plt.legend()
plt.show()

mse = round(sum((results_df['Predicted'] - results_df['Actual'])**2)/len(results_df['Predicted']), 3)
mape = round(sum(abs((results_df['Predicted'] - results_df['Actual'])))/len(results_df['Predicted']), 3)

print("MSE =", mse, "\nMAPE =", mape)

## Air dataset
- https://www.opendatanetwork.com/dataset/datahub.transportation.gov/xgub-n9bw

In [ ]:
carrier_raw = pd.read_csv("carrier_passengers.csv")
carrier_df = carrier_raw[carrier_raw['Year'] == 2023].sort_values(by = 'data_dte').reset_index(drop = True)
carrier_df = carrier_df.drop(columns=["data_dte", "usg_apt_id", "usg_wac", "fg_apt_id", "fg_wac", "airlineid", "type", "Scheduled", "Charter"])

carrier_df

### Numeric encoding
- Tried one-hot and PCA, didn't work well
    - Create the encoding for the flight and carrier, and use pca to determine which columns are most useful in determining the total

In [ ]:
carrier_df

In [ ]:
columns_to_encode = ['usg_apt', 'fg_apt', 'carrier']
temp_df = carrier_df.copy()
temp_df = temp_df[columns_to_encode]

## Scale variables
standard_scaler = StandardScaler()
model_df = pd.DataFrame(standard_scaler.fit_transform(carrier_df.drop(columns = columns_to_encode)))
model_df = pd.concat([model_df, temp_df], axis=1)

## Construct encoders
le_usg_apt = LabelEncoder()
le_fg_apt = LabelEncoder()
le_carrier = LabelEncoder()

## Encode the data
model_df['usg_apt_encoded'] = le_usg_apt.fit_transform(model_df['usg_apt'])
model_df['fg_apt_encoded'] = le_fg_apt.fit_transform(model_df['fg_apt'])
model_df['carrier_encoded'] = le_carrier.fit_transform(model_df['carrier'])
model_df = model_df.drop(columns=['usg_apt', 'fg_apt', 'carrier']).rename(columns = {0: "Year", 1:"Month", 2:"carriergroup", 3:"Total"})

# ## Conduct PCA
# pca = PCA()
# pca.fit(one_hot_scaled)
# one_hot_pca = pca.transform(one_hot_scaled)
# explained_var = pca.explained_variance_ratio_

# # ## Remove low variability
# # one_hot.loc[-1] = list(explained_var)
# # columns_to_drop = one_hot.columns[one_hot.iloc[-1] < (max(list(explained_var)) + min(list(explained_var)))/2]
# # one_hot = one_hot.drop(columns=columns_to_drop)
# # one_hot = one_hot.drop(-1)
# # one_hot.head()

model_df.head()

### Fit the model

In [ ]:
air_model_df = model_df.copy()

target = air_model_df['Total']
predictors = air_model_df.drop(columns=['Total', "Year"])

## Format to TimeSeries
past = 24
future = 24
predictors_ts = TimeSeries.from_dataframe(predictors)
predictors_ts = predictors_ts.astype('float32')

model = NHiTSModel(input_chunk_length = past, output_chunk_length = 5)
model.fit(predictors_ts, epochs=100)

## Predict on self
pred = model.predict(n = len(air_model_df['Month']))

In [ ]:
## Plot predict v train
# results_df = pd.DataFrame(standard_scaler.inverse_transform(pred.values()))
# results_df = pd.concat([carrier_df['Month'], results_df, \
#                         target], axis = 1)
# results_df = results_df[['Month', 'Predicted', 'Total']]
# results_df = results_df.groupby(by = 'Month').sum().reset_index()

# plt.plot(results_df['Month'], results_df['Predicted'], label = 'Predicted')
# plt.plot(results_df['Month'], results_df['Actual'], label = 'Actual')
# plt.legend()
# plt.show()

# mse = round(sum((results_df['Predicted'] - results_df['Actual'])**2)/len(results_df['Predicted']), 3)
# mape = round(sum(abs((results_df['Predicted'] - results_df['Actual'])))/len(results_df['Predicted']), 3)

# print("MSE =", mse, "\nMAPE =", mape)
pred.values()